In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from datetime import datetime

spark = SparkSession.builder.appName("CustomerData").getOrCreate()


spark.sql("CREATE SCHEMA IF NOT EXISTS firstretail.customer_data")


customer_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("region", StringType(), True),
    StructField("status", StringType(), True),
    StructField("signup_date", DateType(), True)
])


customer_data = [
    (1, "Alice Johnson", "West", "Active",datetime(2023,1,15)),
    (2, "Bob Smith", "East", "Inactive",datetime(2023,2,20)),
    (3, "Charlie Lee", "West", "Active", datetime(2023,3,10))
]

customers_df = spark.createDataFrame(customer_data, schema=customer_schema)
display(customers_df)
customers_df.write.format("delta").mode("overwrite").saveAsTable("firstretail.customer_data.customers")
spark.sql("SELECT customer_id, name, region FROM firstretail.customer_data.customers WHERE status = 'Active' AND region = 'West'")

customer_id,name,region,status,signup_date
1,Alice Johnson,West,Active,2023-01-15
2,Bob Smith,East,Inactive,2023-02-20
3,Charlie Lee,West,Active,2023-03-10


DataFrame[customer_id: int, name: string, region: string]

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

spark = SparkSession.builder.appName("InventoryUpdates").getOrCreate()

spark.sql("CREATE SCHEMA IF NOT EXISTS ecommerce.inventory")


inventory_schema = StructType([
    StructField("product_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("stock", IntegerType(), True),
    StructField("warehouse", StringType(), True)
])


initial_data = [
    (101, "Laptop", 50, "A"),
    (102, "Smartphone", 100, "B")
]
initial_df = spark.createDataFrame(initial_data, schema=inventory_schema)

initial_df.write.format("delta").mode("overwrite").saveAsTable("ecommerce.inventory.products")


update_data = [
    (101, "Laptop", 45, "A"),
    (103, "Tablet", 30, "A")
]
update_df = spark.createDataFrame(update_data, schema=inventory_schema)



In [0]:
from delta.tables import DeltaTable

delta_inventory = DeltaTable.forName(spark, "ecommerce.inventory.products")


delta_inventory.alias("target").merge(
    update_df.alias("source"),
    "target.product_id = source.product_id"
).whenMatchedUpdate(
    set={"stock": "source.stock"}
).whenNotMatchedInsert(
    values={
        "product_id": "source.product_id",
        "name": "source.name",
        "stock": "source.stock",
        "warehouse": "source.warehouse"
    }
).execute()

final_df = spark.sql("""
    SELECT *
    FROM ecommerce.inventory.products
    ORDER BY product_id
""")
final_df.show()


+----------+----------+-----+---------+
|product_id|      name|stock|warehouse|
+----------+----------+-----+---------+
|       101|    Laptop|   45|        A|
|       102|Smartphone|  100|        B|
|       103|    Tablet|   30|        A|
+----------+----------+-----+---------+

